In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(train_data, test_data), ds_info = tfds.load('tf_flowers',
                                             split=['train[:80%]', 'train[80%:]'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

In [ ]:
train_data = train_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 266ms/step - accuracy: 0.2796 - loss: 1.8229 - val_accuracy: 0.3965 - val_loss: 1.5450
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.3737 - loss: 1.4994 - val_accuracy: 0.4155 - val_loss: 1.4422
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.4009 - loss: 1.4395 - val_accuracy: 0.4346 - val_loss: 1.3818
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.4099 - loss: 1.4034 - val_accuracy: 0.4687 - val_loss: 1.3505
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 0.4356 - loss: 1.3651 - val_accuracy: 0.4414 - val_loss: 1.3493
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.4394 - loss: 1.3384 - val_accuracy: 0.4210 - val_loss: 1.3448
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 112ms/step - accuracy: 0.4597 - loss: 1.3174 - val_accuracy: 0.4169 - val_loss: 1.3470
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.4745 - loss: 1.2940 - val_accu

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
fine_tune_history = model.fit(train_data,
                              validation_data=test_data,
                              epochs=15)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 101s 579ms/step - accuracy: 0.9998 - loss: 0.0010 - val_accuracy: 0.7507 - val_loss: 3.9823
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 39s 316ms/step - accuracy: 0.9989 - loss: 0.0084 - val_accuracy: 0.7943 - val_loss: 3.2991
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 317ms/step - accuracy: 0.9877 - loss: 0.0769 - val_accuracy: 0.7943 - val_loss: 3.6372
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 312ms/step - accuracy: 0.9940 - loss: 0.0256 - val_accuracy: 0.8134 - val_loss: 3.0562
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 319ms/step - accuracy: 0.9995 - loss: 0.0066 - val_accuracy: 0.8174 - val_loss: 2.9001
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 319ms/step - accuracy: 0.9986 - loss: 0.0033 - val_accuracy: 0.8229 - val_loss: 2.7808
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 317ms/step - accuracy: 1.0000 - loss: 5.3569e-04 - val_accuracy: 0.8243 - val_loss: 2.7717
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 312ms/step - accuracy: 0.9982 - loss: 0.0080 - val

In [ ]:
test_loss, test_acc = model.evaluate(test_data)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.8260 - loss: 2.5708


In [ ]:
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred = np.argmax(model.predict(test_data), axis=1)

23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 248ms/step


In [ ]:
print(classification_report(y_true, y_pred, target_names=ds_info.features['label'].names))

              precision    recall  f1-score   support

   dandelion       0.84      0.86      0.85       159
       daisy       0.78      0.84      0.81       120
      tulips       0.84      0.82      0.83       186
  sunflowers       0.89      0.76      0.82       138
       roses       0.76      0.82      0.79       131

    accuracy                           0.82       734
   macro avg       0.82      0.82      0.82       734
weighted avg       0.83      0.82      0.82       734

